# Part 3: Sensitivity analysis for β = 0.90, 0.95, 0.99

_Replace this with your work._

In [ ]:
# Your code starts here
# Part 3: CVaR-minimizing portfolios for beta in {0.90, 0.95, 0.99}
import os, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import linprog

BETA_LIST = [0.90, 0.95, 0.99]
R_TARGET = 0.0002
INDEX_COL = "NDX"

def _preferred_path(fname: str) -> str:
    if os.path.exists(fname): return fname
    alt = os.path.join("/mnt/data", fname)
    return alt if os.path.exists(alt) else fname

prices_2019_path = _preferred_path("../data/stocks2019.csv")
prices_2020_path = _preferred_path("../data/stocks2020.csv")  # not used here

# Robust datetime indexer
def ensure_datetime_index(df: pd.DataFrame) -> pd.DataFrame:
    cand_names = {"date","timestamp","time","day","tradingday"}
    date_col = None
    for c in df.columns:
        if c.lower() in cand_names:
            date_col = c
            break
    if date_col is None:
        # fall back: try to parse the FIRST column as dates
        first = df.columns[0]
        try:
            _ = pd.to_datetime(df[first])
            date_col = first
        except Exception:
            raise ValueError(
                "Could not find/parse a date column. "
                f"Available columns: {list(df.columns)}. "
                "Rename your date column to 'Date' or ensure the first column is a parseable date."
            )
    out = df.copy()
    out[date_col] = pd.to_datetime(out[date_col], errors="coerce")
    out = out.dropna(subset=[date_col]).sort_values(date_col).set_index(date_col)
    return out

def returns_from_prices(prices: pd.DataFrame) -> pd.DataFrame:
    prices = ensure_datetime_index(prices)
    prices = prices.select_dtypes(include=[np.number])
    return prices.pct_change().dropna(how="any")

def solve_cvar_lp(returns: pd.DataFrame, beta: float, R_target: float):
    Y = returns.values; q, n = Y.shape
    mu = returns.mean(0).values
    N = n + 1 + q
    i_alpha = n; i_u0 = n+1
    c = np.zeros(N); c[i_alpha] = 1.0; c[i_u0:i_u0+q] = 1.0/((1.0-beta)*q)
    bounds = [(0,None)]*n + [(None,None)] + [(0,None)]*q
    A_ub, b_ub = [], []
    for k in range(q):
        row = np.zeros(N); row[:n] = -Y[k,:]; row[i_alpha] = -1.0; row[i_u0+k] = -1.0
        A_ub.append(row); b_ub.append(0.0)
    row = np.zeros(N); row[:n] = -mu; A_ub.append(row); b_ub.append(-R_target)
    A_ub = np.vstack(A_ub); b_ub = np.array(b_ub)
    A_eq = np.zeros((1,N)); A_eq[0,:n] = 1.0; b_eq = np.array([1.0])
    res = linprog(c, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq,
                  bounds=bounds, method="highs")
    if not res.success: raise RuntimeError(f"LP failed: {res.message}")
    z = res.x
    return {"x": z[:n], "alpha": z[i_alpha], "objective_cvar": float(c @ z), "status": "optimal", "solver": res}

def empirical_cvar_of_x(returns: pd.DataFrame, x: np.ndarray, beta: float) -> float:
    losses = -(returns.values @ x)
    var = np.quantile(losses, beta, method="higher")
    tail = losses[losses >= var]
    return float(tail.mean()) if tail.size else float(var)

prices_2019 = pd.read_csv(prices_2019_path)
rets19_all = returns_from_prices(prices_2019)

if INDEX_COL not in rets19_all.columns:
    raise ValueError(f"Expected index column '{INDEX_COL}' in CSV. Got: {list(rets19_all.columns)[:5]}...")

assets = [c for c in rets19_all.columns if c != INDEX_COL]
R19 = rets19_all[assets]

solutions = {}
for beta in BETA_LIST:
    sol = solve_cvar_lp(R19, beta=beta, R_target=R_TARGET)
    sol["empirical_cvar_2019"] = empirical_cvar_of_x(R19, sol["x"], beta)
    solutions[beta] = sol

summary = pd.DataFrame({
    "beta": BETA_LIST,
    "LP_objective_CVaR_2019": [solutions[b]["objective_cvar"] for b in BETA_LIST],
    "Empirical_CVaR_2019":     [solutions[b]["empirical_cvar_2019"] for b in BETA_LIST],
})
display(summary)

weights_df = pd.DataFrame({f"β={b:.2f}": solutions[b]["x"] for b in BETA_LIST}, index=assets)
ax = weights_df.plot(kind="bar", figsize=(13,5))
ax.set_title("Part 3 — Portfolio Weights vs β (trained on 2019)")
ax.set_ylabel("Weight"); ax.legend(); plt.tight_layout(); plt.show()
